# Homework 5-2: Developing a code for carbonate systems
## Abby Baskind
## 5 October 2023

hi gabby

In [ ]:
from matplotlib import pyplot as plt
from matplotlib.dates import DateFormatter
import numpy as np
import xarray as xr
import pandas as pd
import scipy
from datetime import datetime, timedelta
import time
import seaborn
import matplotlib.dates as mdates
import bottleneck as bn
import PyCO2SYS as pyco2
import gsw
import math
import netCDF4 as nc
import requests

# Import K's code for calculating the coefficients of the carb system
import calc_coeffs as co2
import H_poly as hpoly
from importlib import reload
import warnings
warnings.filterwarnings('ignore')
import cartopy.crs as ccrs
import cartopy.mpl.ticker as cticker

In [93]:
def pHsolver(TA = None, temperature = None, salinity = None, pCO2 = None, pHlo = None, pHhi = None):
    import calc_coeffs as co2
    import H_poly as hpoly
    import numpy as np 
    import math
    
    """
    Inputs:
        - total alkalinity in umol/kg
        - pCO2 in uatm
        - temperature in °C
        - salinity in PSU
        - pHlo and pHhi in total scale
    
    Returns:
        - [H+] in umol/kg
        - pH in total scale
        - [HCO3] in umol/kg
        - [CO3] in umol/kg
        - [CO2*] in umol/kg
        - Csat = [HCO3] + [CO3] + [CO2*] in umol/kg
    """
    
    
    # Check that all the inputs are there
    # I'm using keyword arguments because I think this
    # will make use easier in the future as we have more
    # diverse inputs
    if TA == None:
        print('Bitch, provide total alkalinity.')
        return
    elif temperature == None:
        print('Give me temperature.')
        return
    elif salinity == None:
        print('Where the hell is your salinity?')
        return
    elif pCO2 == None:
        print('So no pCO2? Get out of here.')
        return
    elif pHlo == None or pHhi == None:
        print('Give me pHlo and pHhi!!!')
        return
    
    TA = TA*1e-6
    pCO2 = pCO2*1e-6
    # Calculate the coefficients
    coeffs = co2.calc_coeffs(temperature, salinity)
    k0 = coeffs['k0']
    k1 = coeffs['k1']
    k2 = coeffs['k2']
    kb = coeffs['kb']
    kw = coeffs['kw']
    BT = coeffs['BT']
    
    # Calculate CO2* from pCO2
    # K0 = [CO2*]/pCO2
    co2star = k0 * pCO2
    
    # Based on pH limits, convert to [H+] limits
    Hhi = 10**(-pHlo)
    phlo = 6
    Hlo = 10**(-pHhi)
    Hmid = np.mean([Hhi,Hlo])
    threshold = Hhi-Hlo
    while np.abs(threshold) > 5e-23:
        Hmid = np.mean([Hhi,Hlo])
        sol = hpoly.H_poly(Hmid, TA, co2star, k0, k1, k2, kb, BT)
        if sol > 0:
            Hhi = Hmid
        elif sol < 0:
            Hlo = Hmid
        if Hlo > Hhi:
            print('mix up')
            break
        threshold = Hhi-Hlo
    Hmid = np.mean([Hhi,Hlo])
    H = Hmid
    pH = -math.log10(H)
    
    # K1 = [HCO3][H]/[CO2*]
    HCO3 = (k1 * co2star)/H
    # k2 = [CO3][H]/[HCO3]
    CO3 = (k2 * HCO3)/H
    # Csat = CO3 + CO2* + HCO3
    Csat = CO3 + co2star + HCO3
    
    # Return all the data in a dictionary 
    data = {
        '[H+]': H/1e-6,
        'pH': pH,
        '[HCO3]': HCO3/1e-6,
        '[CO3]': CO3/1e-6,
        '[CO2*]': co2star/1e-6,
        'Csat': Csat/1e-6,
        }
    return data
        

## Note that the inputs and outputs of my function are in umol/kg

In [95]:
results = pHsolver(TA = 2350, temperature = 20, salinity = 34.5, pCO2 = 380, pHlo = 7, pHhi = 9)
results

{'[H+]': 0.008628886017489654,
 'pH': 8.064045267747677,
 '[HCO3]': 1871.5772430896734,
 '[CO3]': 197.68372333702308,
 '[CO2*]': 12.348181085475977,
 'Csat': 2081.6091475121725}